In [1]:
import pandas as pd

df = pd.read_csv('data_dac.csv', sep = ',', header = 0, error_bad_lines=False, date_parser=['posted_at'])
df = df[df.named_id.notnull() == True ]
df.nb_follower = df.nb_follower.astype('int32')
df.nb_following = df.nb_following.astype('int32')
df.length = df.length.astype('int32')
df.orthographe = df.orthographe.astype('float64')

b'Skipping line 75: expected 17 fields, saw 18\nSkipping line 106: expected 17 fields, saw 18\nSkipping line 138: expected 17 fields, saw 18\nSkipping line 165: expected 17 fields, saw 18\nSkipping line 166: expected 17 fields, saw 19\nSkipping line 187: expected 17 fields, saw 19\nSkipping line 261: expected 17 fields, saw 18\nSkipping line 269: expected 17 fields, saw 19\nSkipping line 339: expected 17 fields, saw 19\nSkipping line 702: expected 17 fields, saw 19\nSkipping line 731: expected 17 fields, saw 18\nSkipping line 773: expected 17 fields, saw 18\nSkipping line 844: expected 17 fields, saw 19\nSkipping line 1262: expected 17 fields, saw 18\nSkipping line 1293: expected 17 fields, saw 18\nSkipping line 1323: expected 17 fields, saw 20\nSkipping line 1577: expected 17 fields, saw 18\nSkipping line 1685: expected 17 fields, saw 19\nSkipping line 1967: expected 17 fields, saw 18\nSkipping line 1973: expected 17 fields, saw 18\nSkipping line 2102: expected 17 fields, saw 18\nSkip

Dans un second temps nous allons procéder à des analyses sur les textes des tweets pour voir si des tendances se dégagent et ce que l'on peut en tirer.

Nous allons commencer par définir une fonction permettant de tokenizer les tweets que nous souhaitons analyser. Nous allons d'abord supprimer tout les hyperliens et mention d'utilisateur qui risque de perturber notre analyse. Nous allons aussi supprimer les stopwords et découper les hastags en fonction des majuscules (#GiletJaune devient 'Gilet Jaune')

In [3]:
import re
import fr_core_news_md
nlp = fr_core_news_md.load()

from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from Emojilist import emojilist

In [5]:


stopwords_fr = stopwords.words('french')


def tokenize(message):
        message = re.sub(r'(?:\@|https?\://)\S+', '', message)  # remove links and @username
        message = re.sub(r'[^\w\s]', ' ', message)  # remove non-alphanumeric characters
        doc = nlp (message)
        stopWords = new_stopwords
        hashtag = False
        list = [str(token) for token in doc]
        tokens = []
        for elt in list :
            #remove stopwords
            if elt not in stopWords :
                #couper les lettres qui se repètent plus de 2fois de suite
                for i in range(len(elt)-1,1, -1):
                    if elt[i] == elt [i-1] and elt[i] == elt [i-2] :
                        elt = elt[:i]+elt[i+1:]
                #On coupe le mot si il s'agit du terme après un hastag
                if hashtag == True :
                    decoup = re.findall('[A-Z][^A-Z]*',elt)
                    for word in decoup :
                        tokens.append(word)
                    hashtag = False
                elif elt == '#' :
                    hashtag = True
                elif elt in emojilist :
                    pass
                else :
                    tokens.append(elt)
        return tokens

In [6]:
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
import collections



def cluster_texts_Kmeans( texts, k=5):
        """ Transform texts to Tf-Idf coordinates and cluster texts using K-Means """
        vectorizer = TfidfVectorizer(
            tokenizer=tokenize,
            max_features=1500,
            max_df=0.7,
            min_df=5,
            lowercase=True)

        tfidf_model = vectorizer.fit_transform(texts)
        km_model = KMeans(n_clusters=k, init='k-means++', n_init=10)
        km_model.fit_predict(tfidf_model)
        
        clustering = collections.defaultdict(list)

        print("Top terms per cluster:")
        order_centroids = km_model.cluster_centers_.argsort()[:, ::-1]
        #print(order_centroids)
        terms = vectorizer.get_feature_names()
        for i in range(k):
            print("\n Cluster %d:" % i)
            for ind in order_centroids[i, :10]:
                print(' %s' % terms[ind])


        

In [7]:
print(cluster_texts_Kmeans(df.text))

Top terms per cluster:

 Cluster 0:
   
 bien
 

 quoi
 via
 ah
 veux
 aime
 rien
 faut

 Cluster 1:
   
 autres
 dis
 pseudo
 nouveau
 3
 oeil
 parfait
 pouvez
 changer

 Cluster 2:
 faire
   
 vais
 faut
 plaisir
 rien
 partie
 aller
 genre
 envie

 Cluster 3:
 sais
   
 bien
 demande
 imagine
 fou
 cours
 xd
 mdr
 gens

 Cluster 4:
 bien
 

 mdrr
 cette
 gens
 faut
 mdr
 jamais
 quoi
 2
None


Nous remarquons qu'il reste beaucoup de stopwords dans les mots les plus fréquents. Nous allons donc compléter la liste de stopwords pour qu'elle soit plus complète que la liste de base présente dans ntlk

In [4]:
new_stopwords = stopwords.words('french') + ['oui','non','merci','a','bah','toujours','va','dit','alors','juste','sujet','au','la','sur','aucuns','le','ta','aussi','les','tandis',
'autre','leur','tellement','avant','là','tels','avec','ma','tes','avoir','maintenant','ton','bon','mais','tous',
'car','mes','tout','ce','mine','trop','cela','moins','très','ces','mon','tu','ceux','mot','voient','chaque','même','vont',
'ci','ni','votre','comme','nommés','vous','comment','notre','vu','dans','nous','ça','des','ou','étaient','du','où','état',
'dedans','par','étions','dehors','parce','été','depuis','pas','être','devrait','peut','doit','peu','donc','plupart',
'dos','pour','début','pourquoi','elle','quand','elles','que','en','quel','encore','quelle','essai','quelles','est','quels',
'et','qui','eu','sa','fait','sans','faites','ses','fois','seulement','font','si','hors','sien','ici','son','il','sont',
'ils','sous','je','soyez','plus']


In [16]:
print(cluster_texts_Kmeans(df.text))

Top terms per cluster:
Cluster 0:
 plus
   
 non
 vraiment
  

 sait
 france
 5
 jamais
 actuellement
Cluster 1:
 bien
   
 sûr
 sais
 toutes
 deux
  

 ah
 connu
 aime
Cluster 2:
   
 3
 via
 ah
 jamais
 no
 malaise
 live
 père
 pareil
Cluster 3:
 

 non
   
  

 00
 
 
 mdr
 bonjour
 dis
 peux
Cluster 4:
   
 faire
 quoi
 faut
 cette
 mdrr
 mdr
 2
 sais
 rien
None


Maintenant on obtient des termes plus pertinents pour qualifier les clusters. Néanmoins il nous reste donc a définir le nombre de cluster qu'il serait pertinent de faire car pour l'instant il est fixé par nous.

Pour cela nous allons utiliser la méthode DBSCAN

In [9]:
from sklearn.cluster import DBSCAN

def cluster_texts_DBSCAN( texts):
        vectorizer = TfidfVectorizer(
            tokenizer=tokenize,
            max_features=1500,
            max_df=0.7,
            min_df=5,
            lowercase=True)

        tfidf_model = vectorizer.fit_transform(texts)
        km_model = DBSCAN(eps=0.5, min_samples=5)
        print(tokenize(texts[0]))
        print(tfidf_model)
        km_model.fit_predict(tfidf_model)
        


In [10]:
print(cluster_texts_DBSCAN(df.text))

['RT', ' ', 'frenchweb', '  ', 'INSIDERS', ' ', 'Robots', 'livreurs', '  ', 'SoftBank', 'mise', 'près', 'milliard', 'dollars', 'Nuro', ' ']
  (0, 554)	0.6058126965772836
  (0, 9)	0.35353530607209843
  (0, 510)	0.7127438277714285
  (1, 245)	0.44840990704972183
  (1, 660)	0.6555957974788901
  (1, 498)	0.6075546934948969
  (2, 9)	0.5815238990989503
  (2, 365)	0.8135293201703022
  (3, 381)	0.7259073321058642
  (3, 137)	0.6877925160940229
  (4, 451)	0.548550952226257
  (4, 657)	0.4609726875835015
  (4, 475)	0.4543181679508845
  (4, 216)	0.5293307438487418
  (5, 634)	0.5385900560495686
  (5, 86)	0.5957855115410758
  (5, 306)	0.5957855115410758
  (6, 247)	0.4750147246387601
  (6, 606)	0.6995081379342027
  (6, 164)	0.5339001557783881
  (7, 9)	0.24812021320330557
  (7, 130)	0.9687292499971012
  (8, 9)	0.37405240896457803
  (8, 6)	0.44890291513159025
  (8, 350)	0.5122751511966681
  :	:
  (3346, 327)	0.32197869053399153
  (3346, 303)	0.3933505360740032
  (3346, 1)	0.34497017433208493
  (3347, 222